In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import sys
sys.path.append('../../PyCurb/')
sys.path.append('..')

In [3]:
import os
import json
from os.path import join
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime, time
from copy import deepcopy
from IPython.display import Image
from pprint import pprint
from pdb import set_trace

from ccd.utils import read_layers_from_gdb, day_range
from ccd.parking_time_range import ParkingTimeRange
from curblr.authority import Authority
from curblr import (CurbLRObject, Regulation, Location, Feature, FeatureCollection, 
                    TimeSpan, Rule, UserClass, Payment, Manifest)
from curblr.utils import parse_time, to_camelcase, from_camelcase, shift_days
from curblr.time_rule import DaysOfWeek, TimeOfDay, DesignatedPeriod
from curblr.constants import DAYS
from curblr.parking_time import ParkingTime, rule_parking_time
from curblr.parking_time_range import RuleParkingTimeRange, HourParkingTimeRange

In [4]:
def row_to_ptrs(r):
    ranges = []
    for i in range(1,3):
        dr = day_range(r['Day_From_{}'.format(i)], r['Day_To_{}'.format(i)])
        
        ptr = ParkingTimeRange(
            rule_parking_time(r['Time_From_{}'.format(i)]), 
            rule_parking_time(r['Time_To_{}'.format(i)]))
        
        if ptr.start and ptr.end:
            ptr.start = ptr.start.time()
            ptr.end = ptr.end.time()
            if ptr.end.hour == 0 and ptr.end.minute == 0:
                ptr.end = time(23, 59)
            
            if ptr.start.hour > ptr.end.hour:
                ptr1 = ParkingTimeRange(
                    ParkingTime(ptr.start.hour, ptr.start.minute),
                    ParkingTime(23, 59)
                )
                ranges.append((dr, ptr1))

                ptr2 = ParkingTimeRange(
                    ParkingTime(0 ,0),
                    ParkingTime(ptr.end.hour, ptr.end.minute)
                )
                ranges.append((shift_days(dr), ptr2))
            elif not dr and ptr.all_time:
                ranges.append(None)
            else:
                ranges.append((dr, ptr))
    
    return tuple(ranges)

def plot_img(guid):
    return Image(filename='../img/{}.png'.format(guid.replace('{', '').replace('}', ''))) 

In [6]:
geodatabase = '../data/raw/phase1/ParkingZones.gdb/'
layers = read_layers_from_gdb(geodatabase)
pz = layers['Parking_Zones'].to_crs({'init':'epsg:4326'})
pzr = layers['Parking_Zone_Regulations']

In [7]:
with open('../data/interim/parking_data/output_30_side-of-street_hierarchy-8_best-direction.matched.geojson') as f:
    lrm = json.load(f)
lrms = {f['properties']['pp_globalid']: f for f in lrm['features']}

In [8]:
with open('../data/interim/reg_lookup.json') as f:
    d = json.load(f)

In [15]:
pzr[pzr['Time_From_1'] == 9910]

,GlobalID,Regulation,Time_From_1,Time_To_1,Day_From_1,Day_To_1,Time_From_2,Time_To_2,Day_From_2,Day_To_2,Notes,GUID,Created_By,Created_On,Edited_By,Edited_On,Time_Limit,RegCat
407,{4B7048D1-CDEC-4AAA-8D82-8FB184CB4F61},No Parking,9910,9910,888,888,9911,9911,N/A,N/A,school hours,{713C8CDA-12A4-4243-A2F6-E6954AF16AB3},FieldEditor4,2019-01-16T12:28:52,DYAW,2019-02-28T12:11:18,NaN,No Parking / Stopping / Standing


In [13]:
phila = Authority('ppa', 'http://www.philapark.org/', '1-888-591-3636')
manifest = Manifest(time_zone='EET', currency='USD', authority=phila)
unlimited_parking = Rule('parking', 'no regulations listed')
fc = FeatureCollection(manifest=manifest)

for i, seg in pz.iterrows():
    
    gid = seg['GlobalID']
        
    iid = gid.replace('{', '').replace('}', '')
    
    images = [join('img', i) for i in os.listdir('../img/') if iid in i]
    
    # initialize the feature
    lrm = lrms.get(gid)
    if lrm:
        lrm['properties']['marker'] = 'sign'
    loc = Location.from_lr_feature(lrm, 
                                   object_id='pp_globalid', 
                                   street_name='pp_street')
    feature = Feature(seg['geometry'], loc, images=images)
    
    # get all regulations for this segment
    regs_gdf = pzr[pzr['GUID'] == gid]
    
    # payment
    m = seg['Metering']
    payment = Payment(methods = [m.lower().strip()]) if m not in (None, 'No Meters') else None
    
    # regulations
    if len(regs_gdf) == 0:
        reg_obj = Regulation(Rule('parking', 'no regulations listed'))
        feature.add_regulation(reg_obj)
    else:
        for _, r in regs_gdf.iterrows():
            reg_c = r['Regulation'].lower().strip()
            if reg_c == 'other (see notes)':
                reg_c = r['Notes']
                if reg_c:
                    reg_c = reg_c.lower().strip()
                else:
                    reg_c = 'no regulations listed'

            # get mapping of reg string to other characteristics
            s = d.get(reg_c)

            # priority
            priority = s.get('priority', 4)

            # rule
            max_stay = r['Time_Limit'] * 60 if not np.isnan(r['Time_Limit']) else None
            
            clr_rule = Rule(activity = s['activity'], 
                            reason = s['reason'], 
                            max_stay=max_stay, 
                            payment=True if payment and s['activity'] == 'parking' else False)

            clr_userclasses = None
            classes = [c for c in s.get('classes') if c]

            if len(classes) > 0:
                clr_userclasses = [UserClass(classes)]
        
            # timespans
            timespans = []

            for p in row_to_ptrs(r):
                if p:
                    dr, ptr = p
                    if dr:
                        if len(dr) < 7:
                            days_of_week = DaysOfWeek(dr)
                        else:
                            days_of_week = None
                    else: 
                        days_of_week = None

                    if ptr.all_time or ptr.invalid:
                        time_of_day = None
                    else:
                        time_of_day = TimeOfDay(ptr.start, ptr.end)

                    # designated period
                    designated_period = None
                    dp = s.get(designated_period)
                    if dp:
                        designated_period = DesignatedPeriod(**dp)
                    elif r['Time_From_1'] == 9910:
                        designated_period = DesignatedPeriod('school hours', 'only during')
                    
                    ts = TimeSpan(days_of_week=days_of_week, 
                                  times_of_day=[time_of_day] if time_of_day else time_of_day, 
                                  designated_periods=[designated_period] if designated_period else designated_period)

                    new_ts = True
                    for t in timespans:
                        if not ts.days_of_week:
                            if not t.days_of_week:
                                continue
                        elif t.days_of_week: 
                            if sorted(ts.days_of_week.days) != sorted(t.days_of_week.days):
                                continue

                        for tod in ts.times_of_day:
                            t.add_time_of_day(tod)
                        new_ts = False

                    if new_ts:
                        timespans.append(ts)
            if timespans == []:
                timespans = None
            
            # generate regulation
            reg_obj = Regulation(clr_rule, 
                                 clr_userclasses, 
                                 timespans, 
                                 payment=payment,
                                 priority=priority)

            feature.add_regulation(reg_obj)
    
    feature.fill_unregulated_time(unlimited_parking)
    fc.add_feature(feature)

    if (i + 1) % 500 == 0:
        print('Completed [{}] of {}'.format(i+1, len(pz)))
print('Completed [{}] of {}'.format(i+1, len(pz)))

> <ipython-input-13-066659c2404a>(68)<module>()
-> for p in row_to_ptrs(r):
(Pdb) l
 63  	
 64  	            # timespans
 65  	            timespans = []
 66  	            import pdb
 67  	            pdb.set_trace()
 68  ->	            for p in row_to_ptrs(r):
 69  	                if p:
 70  	                    dr, ptr = p
 71  	                    if dr:
 72  	                        if len(dr) < 7:
 73  	                            days_of_week = DaysOfWeek(dr)
(Pdb) r
Internal StopIteration
> <ipython-input-13-066659c2404a>(35)<module>()
-> for _, r in regs_gdf.iterrows():
(Pdb) print(r)
GlobalID       {DA2EFDA4-33A7-4F37-8941-A9226C5FE69F}
Regulation                                No Stopping
Time_From_1                                      9911
Time_To_1                                        9911
Day_From_1                                        N/A
Day_To_1                                          N/A
Time_From_2                                      9911
Time_To_2            

BdbQuit: 

In [10]:
fc.to_dict()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiLineString',
    'coordinates': (((-75.15226734079631, 39.949376353862746),
      (-75.1508512286913, 39.949200388895626)),)},
   'properties': {'location': {'shstRefId': '0c3866adeefed1c1711af8d7f01d88d5',
     'sideOfStreet': 'right',
     'shstLocationStart': 6.651501209932993,
     'shstLocationEnd': 128.95423159814615,
     'objectId': '{DFA3C5DA-F650-4CDE-99ED-6A947933418B}',
     'marker': 'sign',
     'streetName': 'CHESTNUT ST'},
    'regulations': [{'rule': {'activity': 'no standing',
       'reason': 'no stopping',
       'payment': False},
      'priority': 4}]}},
  {'type': 'Feature',
   'geometry': {'type': 'MultiLineString',
    'coordinates': (((-75.15084955182841, 39.94924878704493),
      (-75.150956011474, 39.949264346011084)),)},
   'properties': {'location': {'shstRefId': '0c3866adeefed1c1711af8d7f01d88d5',
     'sideOfStreet': 'unknown',
     'shstLocationStart': 128.2292

In [ ]:
fc.save('../data/interim/ccd.curblr.json', add_timestamp=True)

In [ ]:
fc2 = deepcopy(fc)
fc2.features = [f for f in fc2.features if f.location]
fc2.save('../../curb-map/src/assets/data/ccd_NULL-REMOVED.curblr.json', add_timestamp=True)